In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/23 22:20:29 WARN Utils: Your hostname, UTHMAN-PC resolves to a loopback address: 127.0.1.1; using 172.25.64.187 instead (on interface eth0)
22/02/23 22:20:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/23 22:20:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/23 22:20:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/23 22:20:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-23 22:21:25--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.166.241
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.166.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M   981KB/s    in 17m 12s 

2022-02-23 22:38:38 (712 KB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [4]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [6]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [7]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [15]:
!wc -l head.csv

1001 head.csv


In [16]:
import pandas as pd

In [17]:
df_pandas = pd.read_csv('head.csv')

In [18]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [20]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

Integer - 4 bytes
Long - 8 bytes

In [21]:
from pyspark.sql import types

In [22]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [23]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [24]:
df.show(n=20)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [27]:
list(df.schema)

[StructField(hvfhs_license_num,StringType,true),
 StructField(dispatching_base_num,StringType,true),
 StructField(pickup_datetime,TimestampType,true),
 StructField(dropoff_datetime,TimestampType,true),
 StructField(PULocationID,IntegerType,true),
 StructField(DOLocationID,IntegerType,true),
 StructField(SR_Flag,StringType,true)]

In [28]:
df = df.repartition(30)

In [29]:
df.write.parquet('fhvhv/2021/01/')

22/02/23 22:57:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/23 22:57:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/23 22:57:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/23 22:57:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [44]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

22/02/24 00:02:37 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1890138 ms exceeds timeout 120000 ms
22/02/24 00:02:37 WARN SparkContext: Killing executors is not supported by current scheduler.


In [30]:
df = spark.read.parquet('fhvhv/2021/01/')

In [31]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [34]:
from pyspark.sql import functions as F

In [36]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-02 15:13:25|2021-01-02 15:24:00|         237|         107|   null|
|           HV0003|              B02617|2021-01-02 14:00:02|2021-01-02 14:13:26|          79|         255|   null|
|           HV0003|              B02617|2021-01-04 07:15:12|2021-01-04 07:28:18|          49|          25|   null|
|           HV0003|              B02867|2021-01-05 09:10:43|2021-01-05 09:32:00|         223|         151|   null|
|           HV0003|              B02765|2021-01-05 09:01:59|2021-01-05 09:15:51|         131|         265|   null|
|           HV0003|              B02869|2021-01-05 12:49:08|2021-01-05 13:22:27|

In [37]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [38]:
crazy_stuff('B02884')

's/b44'

In [39]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [40]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-02|  2021-01-02|         237|         107|
|  e/a39| 2021-01-02|  2021-01-02|          79|         255|
|  e/a39| 2021-01-04|  2021-01-04|          49|          25|
|  e/b33| 2021-01-05|  2021-01-05|         223|         151|
|  s/acd| 2021-01-05|  2021-01-05|         131|         265|
|  e/b35| 2021-01-05|  2021-01-05|         255|         210|
|  a/b37| 2021-01-01|  2021-01-01|          14|         228|
|  e/acc| 2021-01-04|  2021-01-04|         174|          20|
|  e/b33| 2021-01-01|  2021-01-01|         118|         251|
|  s/b3d| 2021-01-01|  2021-01-01|         231|         164|
|  s/acd| 2021-01-01|  2021-01-01|         213|         213|
|  s/b13| 2021-01-03|  2021-01-03|         127|         169|
|  e/b3c| 2021-01-03|  2021-01-03|         168|         169|
|  e/b38| 2021-01-02|  2

Traceback (most recent call last):                                              
  File "/home/yusufuthman57/bin/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/home/yusufuthman57/bin/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/home/yusufuthman57/bin/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 642, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/home/yusufuthman57/bin/spark-3.0.3-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError


In [42]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [43]:
!head -n 10 head.csv

<module 'pyspark.sql.functions' from '/home/yusufuthman57/bin/spark-3.0.3-bin-hadoop3.2/python/pyspark/sql/functions.py'>